# Organise and Modify games json files

```json
{
    "1": {
        "name": "A game name",
        "nbr_players": 0,
        "nbr_audience": 0,
        "prompt": "A fixed prompt" or null,
        "exclude": [],
        "host_include": [],
        "host_exclude": [],
        "description": null,
        "tips": null,
        "category": "game category name",
        "uid": unique_number
    }, ...
    "game_key": {
        "field_key": field_value,
        "field_key": field_value,
        "field_key": field_value,
        "field_key": field_value,
        "field_key": field_value,
        "field_key": field_value,
        "field_key": field_value,
        "field_key": field_value,
        "field_key": field_value,
        "field_key": field_value,
        "uid": unique_number
    }
},
```

- Sort the games by name games in a set of json files, and set keys accordingly (1 to n)
- Add and remove a key in each game across all json files (to keep consistency)
- Remove all constraints in the game json file for a set of json file
- Set the field_value for a particular field_key in a set of json files

In [95]:
import hashlib
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
from datetime import datetime
from pathlib import Path
from pprint import pprint

In [96]:
root = Path().absolute().parent
root

WindowsPath('c:/Users/etien/PyProjects/ai2/chris')

## Utility functions

In [97]:
def hash_name(str):
    return hashlib.sha256(str.encode('utf-8')).hexdigest()

def load_and_sort(p2json):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
    sorted_list_games_dict = sorted(games.values(), key=lambda g:g['name'])
    for d in sorted_list_games_dict:
        d.update({'uid':hash_name(d['name'])})
    return sorted_list_games_dict

def sort_json_by_game_names(p2json):
    odict = OrderedDict()
    sorted_list_games_dict = load_and_sort(p2json)
    for i, d in enumerate(sorted_list_games_dict):
        odict[str(i+1)] = d
    print(f"    sorted {i+1} games in {p2json.parent.name}/{p2json.name}")
    with open(p2json, 'w') as fp:
        json.dump(odict, fp, indent=4)

def add_field(p2json, field, save_json=False):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
        for k, v in games.items():
            game_dict = v
            game_dict[field] = game_dict.get(field, "")
            games[k] = game_dict
    if save_json:
        with open(p2json, 'w') as fp:
            json.dump(games, fp, indent=4)
    return games

def remove_field(p2json, field, save_json=False):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
        for k, v in games.items():
            game_dict = v
            if field in game_dict.keys():
                del game_dict[field]
                games[k] = game_dict
    if save_json:
        with open(p2json, 'w') as fp:
            json.dump(games, fp, indent=4)
    return games

class show_full_df:
    def __init__(self):
        self.max_rows = pd.options.display.max_rows
        self.max_cols = pd.options.display.max_columns

    def __enter__(self):
        pd.options.display.max_rows = None
        pd.options.display.max_columns = None
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        pd.options.display.max_rows = self.max_rows
        pd.options.display.max_columns = self.max_cols

## Detect list of all json files

In [98]:
p2config = root / 'config'
assert p2config.exists()

config_dirs = [p for p in p2config.iterdir() if p.is_dir()]
print(f"There are {len(config_dirs)} custom config directories in config")
print('\n'.join(f" - {p.name}" for p in config_dirs))

json_files = [p for p in p2config.glob('**/*.json') if p.suffix == '.json']
if len(json_files) != len(set(json_files)): raise RuntimeError(f"{len(json_files)-len(set(json_files))} duplicated files")
print(f"There are {len(json_files)} json files in total in config and the custom config directories")

There are 4 custom config directories in config
 - ai2
 - ai2-show
 - jam
 - zmack
There are 16 json files in total in config and the custom config directories


## Sort json files

Sort all json files present in `config` folder

In [99]:
json_set = json_files

for i, p in enumerate(json_set):
    print(f"{i:2d}. Sorting {p.absolute()}")
    sort_json_by_game_names(p)

 0. Sorting c:\Users\etien\PyProjects\ai2\chris\config\games.json
    sorted 16 games in config/games.json
 1. Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-07.json
    sorted 24 games in ai2/ai2-games-all-2024-07.json
 2. Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-08.json
    sorted 21 games in ai2/ai2-games-all-2024-08.json
 3. Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-09.json
    sorted 26 games in ai2/ai2-games-all-2024-09.json
 4. Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-12.json
    sorted 21 games in ai2/ai2-games-all-2024-12.json
 5. Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2025-db.json
    sorted 32 games in ai2/ai2-games-all-2025-db.json
 6. Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2025-Q1.json
    sorted 32 games in ai2/ai2-games-all-2025-Q1.json
 7. Sorting c:\Users\etien\PyProjects\ai2\chris\confi

## List all games included in all json files

In [5]:
game_set = set()
for f in json_files:
    with open(f,'r') as fh:
        d = json.load(fh)
        game_set = game_set.union(set(g['name'] for g in d.values() ))

for i, gname in enumerate(sorted(list(game_set))):
    print(f"{i:2d}: {gname}")

 0: A Date with Me
 1: Add-on fairy tale
 2: Chain Death
 3: Da Doo Rap Rap
 4: Da Doo Ron Ron
 5: Do You Have a Receipt?
 6: Double Reverse Alphabet
 7: Emotion Roller Coaster
 8: Emotional Floorplan
 9: Everybody Dies
10: Everyone Dies
11: Forward Reverse
12: Four Square
13: Genres
14: Headphones
15: Highlander
16: Irish Drinking Song
17: Last Comic Standing
18: Last Player Standing
19: More or Less
20: Musical Freeze
21: Musical Paper Chase
22: Mutant Complain
23: Mutant Debate
24: No Laughing
25: One Three Five
26: Pillars
27: Props
28: Puppets
29: Questions Only
30: Rap Battle
31: Sit, Stand, Lie
32: Story, Story, Die!
33: Stunt Doubles
34: Suitcase
35: Sure Ding
36: TV Box
37: That Game
38: Three Monkeys
39: Two lines
40: World's Worst


## Add fields in all json

> Fields to be added in all json file for consistency

Try first without saving, then with saving

In [12]:
field2add = "uid"
save_json = False

for p2json in json_files:
    json_content = add_field(p2json, field2add, save_json=save_json);

# if not save_json: pprint(json_content)

## Review all fields accross json in a set of files

Use a reference json file to extract the field_keys

In [6]:
p2refjson = root / 'config/ai2/ai2-games-all-2024-12.json'
with open(p2refjson, 'r') as fp:
    ref_games = json.load(fp)
    
columns = ['json_file', 'json_fname', 'game_key', 'created'] + list(ref_games['1'].keys())
print(columns)

['json_file', 'json_fname', 'game_key', 'created', 'name', 'nbr_players', 'nbr_audience', 'prompt', 'exclude', 'host_include', 'host_exclude', 'description', 'tips', 'category', 'uid']


In [7]:
dates = pd.DataFrame(
    {
        'fname':[f"{p.parent.name}/{p.name}" for p in json_files],
        'create':[datetime.fromtimestamp(p.stat().st_ctime) for p in json_files],
        'modify':[datetime.fromtimestamp(p.stat().st_mtime) for p in json_files]
    }
).sort_values(by='create').reset_index(drop=True)

dates

,fname,create,modify
0,config/games.json,2023-11-13 22:34:52.292274,2025-03-01 21:39:22.173167
1,ai2/ai2-games.json,2023-11-13 22:39:24.773663,2025-03-01 21:51:56.652451
2,jam/jam-games.json,2023-11-13 22:39:44.303316,2025-03-01 21:53:10.237112
3,zmack/games.json,2023-11-15 16:13:59.377662,2025-03-01 21:54:03.580927
4,ai2-show/ai2-games-2024-01-20.json,2024-01-21 18:48:13.619034,2025-03-01 21:39:22.173167
5,ai2/ai2-games-all.json,2024-01-21 19:36:03.136035,2025-03-01 21:39:22.173167
6,ai2-show/ai2-games-2024-05-18-z15.json,2024-05-18 11:43:45.481013,2025-03-01 21:39:22.173167
7,ai2/ai2-games-all-2024-07.json,2024-07-08 12:15:15.936213,2025-03-01 21:39:22.173167
8,ai2/ai2-games-all-2024-08.json,2024-08-05 19:34:47.276161,2025-03-01 21:39:22.173167
9,ai2-show/ai2-games-2024-08-31.json,2024-08-27 18:12:43.116610,2025-03-01 21:39:22.173167


In [8]:
import pandas as pd

df = pd.DataFrame(columns=columns)
for i, p in enumerate(json_files):
    print(f"{i:02d}: Handling {p.absolute()}")
    with open(p, 'r') as fp:
        games = json.load(fp)
    for k, v in games.items():
        cols = ['json_file', 'json_fname', 'game_key', 'created'] + list(v.keys())
        # values = [str(p.absolute()), k] + list(v.values())
        values = [i, f"config/{p.parent.name}/{p.name}", k, datetime.fromtimestamp(p.stat().st_ctime)] + list(v.values())
        row = pd.DataFrame([values], columns=cols, index=[0])
        df = pd.concat([df, row], ignore_index=True)

00: Handling c:\Users\etien\PyProjects\ai2\chris\config\games.json
01: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-07.json
02: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-08.json
03: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-09.json
04: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-12.json
05: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2025-03-01.json
06: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all.json
07: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games.json
08: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-01-20.json
09: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-05-18-z15.json
10: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-08-31.json
11: Handling c:\Users\etien\PyProjects\ai2\chris\config\jam\jam-

C:\Users\etien\AppData\Local\Temp\ipykernel_15628\3372353124.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row], ignore_index=True)


In [9]:
df.sample(3)

,json_file,json_fname,game_key,created,name,nbr_players,nbr_audience,prompt,exclude,host_include,host_exclude,description,tips,category,uid
117,5,config/ai2/ai2-games-all-2025-03-01.json,11,2025-03-01 21:21:30.830773,Everyone Dies,3,0,"3 Locations: \none normal, one exotic, one unr...",[],[],[],None,None,Energy,
146,6,config/ai2/ai2-games-all.json,1,2024-01-21 19:36:03.136035,A Date with Me,0,0,A date with me is like ....,[],[Knick],"[Lenny, Xu]",None,None,,
15,0,config/config/games.json,16,2023-11-13 22:34:52.292274,World's Worst,0,0,"An occupation, animal, or celebrity",[],[],[],None,None,,


In [10]:
coi = ['name', 'prompt', 'category', 'uid','json_file', 'json_fname', 'created']
df[coi].sort_values(by=['name', 'created'], ascending=True)

,name,prompt,category,uid,json_file,json_fname,created
0,A Date with Me,A date with me is like <object/occupation>.,,,0,config/config/games.json,2023-11-13 22:34:52.292274
171,A Date with Me,A date with me is like <object/occupation/anim...,,,7,config/ai2/ai2-games.json,2023-11-13 22:39:24.773663
254,A Date with Me,A date with me is like <object/occupation>.,,,11,config/jam/jam-games.json,2023-11-13 22:39:44.303316
297,A Date with Me,A date with me is like <object/occupation>.,,,13,config/zmack/games.json,2023-11-15 16:13:59.377662
194,A Date with Me,A date with me is like <object/occupation/anim...,NaN,,8,config/ai2-show/ai2-games-2024-01-20.json,2024-01-21 18:48:13.619034
...,...,...,...,...,...,...,...
253,World's Worst,"An occupation, animal, or celebrity.",All Play,,10,config/ai2-show/ai2-games-2024-08-31.json,2024-08-27 18:12:43.116610
86,World's Worst,"An occupation, animal, or celebrity.",All Play,,3,config/ai2/ai2-games-all-2024-09.json,2024-09-09 19:24:04.543999
296,World's Worst,"An occupation, animal, or celebrity.",All Play,,12,config/jam/open-cast-2024-11-games.json,2024-11-02 13:21:44.312375
107,World's Worst,"An occupation, animal, or celebrity.",All Play,,4,config/ai2/ai2-games-all-2024-12.json,2024-11-25 21:49:53.005439


In [11]:
json_id = 4
json_files[json_id].name

'ai2-games-all-2024-12.json'

In [12]:
df.loc[df.json_file == json_id, :].head()

,json_file,json_fname,game_key,created,name,nbr_players,nbr_audience,prompt,exclude,host_include,host_exclude,description,tips,category,uid
87,4,config/ai2/ai2-games-all-2024-12.json,1,2024-11-25 21:49:53.005439,A Date with Me,8,0,"A date with me is like ....\n\nobject, animal,...",[Eddie],[],"[Don, Haili, Harry, Stacey, Xu, Eddie, Knick]",None,None,All Play,
88,4,config/ai2/ai2-games-all-2024-12.json,2,2024-11-25 21:49:53.005439,Chain Death,4,0,"Occupation, location, ordinary object\n\nAll w...","[Don, Stacey]","[Knick, Etienne, Chris, Yirong, Grace]","[Don, Haili, Harry, Stacey, Xu, Punk]",None,None,Guessing,
89,4,config/ai2/ai2-games-all-2024-12.json,3,2024-11-25 21:49:53.005439,Emotion Roller Coaster,4,0,Four emotions:\n3 emotions of increasing inten...,[Eddie],[],"[Don, Haili, Harry, Stacey, Xu, Eddie]",None,None,Emotion,
90,4,config/ai2/ai2-games-all-2024-12.json,4,2024-11-25 21:49:53.005439,Emotional Floorplan,3,0,Three Emotions of increasing intensity\n\n+ A ...,[Eddie],[],"[Don, Haili, Harry, Stacey, Xu]",None,None,Emotion,
91,4,config/ai2/ai2-games-all-2024-12.json,5,2024-11-25 21:49:53.005439,Everyone Dies,3,0,"3 Locations: \none normal, one exotic, one unr...","[Charles, Don, Elva, Fulvio, Harry, Jennel, Me...","[Etienne, Yirong, Grace]","[Don, Haili, Harry, Stacey, Xu]",None,None,Energy,


### Add Unique Identification 

To add a UID, we will:
- use a unique hash on the game name for each row in the DF
- replace the UID for those games where the name was modified with time, but should have the same UID

Update the latest information for prompt, category, description and tips

Create the UID based on a hash

In [13]:
df.uid = df.name.apply(lambda s:hashlib.sha256(s.encode('utf-8')).hexdigest())
df[coi].sort_values(by=['name', 'created'], ascending=True)


,name,prompt,category,uid,json_file,json_fname,created
0,A Date with Me,A date with me is like <object/occupation>.,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,0,config/config/games.json,2023-11-13 22:34:52.292274
171,A Date with Me,A date with me is like <object/occupation/anim...,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,7,config/ai2/ai2-games.json,2023-11-13 22:39:24.773663
254,A Date with Me,A date with me is like <object/occupation>.,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,11,config/jam/jam-games.json,2023-11-13 22:39:44.303316
297,A Date with Me,A date with me is like <object/occupation>.,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,13,config/zmack/games.json,2023-11-15 16:13:59.377662
194,A Date with Me,A date with me is like <object/occupation/anim...,NaN,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,8,config/ai2-show/ai2-games-2024-01-20.json,2024-01-21 18:48:13.619034
...,...,...,...,...,...,...,...
253,World's Worst,"An occupation, animal, or celebrity.",All Play,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,10,config/ai2-show/ai2-games-2024-08-31.json,2024-08-27 18:12:43.116610
86,World's Worst,"An occupation, animal, or celebrity.",All Play,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,3,config/ai2/ai2-games-all-2024-09.json,2024-09-09 19:24:04.543999
296,World's Worst,"An occupation, animal, or celebrity.",All Play,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,12,config/jam/open-cast-2024-11-games.json,2024-11-02 13:21:44.312375
107,World's Worst,"An occupation, animal, or celebrity.",All Play,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,4,config/ai2/ai2-games-all-2024-12.json,2024-11-25 21:49:53.005439


Check consistency:
- number of unique names
- number of unique uid

In [14]:
print(sorted(df.name.unique()))
df.name.unique().shape, df.uid.unique().shape

['A Date with Me', 'Add-on fairy tale', 'Chain Death', 'Da Doo Rap Rap', 'Da Doo Ron Ron', 'Do You Have a Receipt?', 'Double Reverse Alphabet', 'Emotion Roller Coaster', 'Emotional Floorplan', 'Everybody Dies', 'Everyone Dies', 'Forward Reverse', 'Four Square', 'Genres', 'Headphones', 'Highlander', 'Irish Drinking Song', 'Last Comic Standing', 'Last Player Standing', 'More or Less', 'Musical Freeze', 'Musical Paper Chase', 'Mutant Complain', 'Mutant Debate', 'No Laughing', 'One Three Five', 'Pillars', 'Props', 'Puppets', 'Questions Only', 'Rap Battle', 'Sit, Stand, Lie', 'Story, Story, Die!', 'Stunt Doubles', 'Suitcase', 'Sure Ding', 'TV Box', 'That Game', 'Three Monkeys', 'Two lines', "World's Worst"]


((41,), (41,))

### Identify the indexes of those games that should share the same UID

In [15]:
coi

['name', 'prompt', 'category', 'uid', 'json_file', 'json_fname', 'created']

In [16]:
for gname in sorted(df.name.unique()):
    with show_full_df():
        display(df.loc[df.name == gname, coi].sort_values(by='created', ascending=True))

,name,prompt,category,uid,json_file,json_fname,created
0,A Date with Me,A date with me is like <object/occupation>.,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,0,config/config/games.json,2023-11-13 22:34:52.292274
171,A Date with Me,A date with me is like <object/occupation/anim...,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,7,config/ai2/ai2-games.json,2023-11-13 22:39:24.773663
254,A Date with Me,A date with me is like <object/occupation>.,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,11,config/jam/jam-games.json,2023-11-13 22:39:44.303316
297,A Date with Me,A date with me is like <object/occupation>.,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,13,config/zmack/games.json,2023-11-15 16:13:59.377662
194,A Date with Me,A date with me is like <object/occupation/anim...,NaN,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,8,config/ai2-show/ai2-games-2024-01-20.json,2024-01-21 18:48:13.619034
146,A Date with Me,A date with me is like ....,,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,6,config/ai2/ai2-games-all.json,2024-01-21 19:36:03.136035
212,A Date with Me,A date with me is like ....,All Play,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,9,config/ai2-show/ai2-games-2024-05-18-z15.json,2024-05-18 11:43:45.481013
16,A Date with Me,A date with me is like ....,All Play,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,1,config/ai2/ai2-games-all-2024-07.json,2024-07-08 12:15:15.936213
40,A Date with Me,A date with me is like ....,All Play,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,2,config/ai2/ai2-games-all-2024-08.json,2024-08-05 19:34:47.276161
233,A Date with Me,A date with me is like ....,All Play,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...,10,config/ai2-show/ai2-games-2024-08-31.json,2024-08-27 18:12:43.116610


,name,prompt,category,uid,json_file,json_fname,created
147,Add-on fairy tale,None,,9e5ffe83e1eb131d9d2e3970e64a72d0736020e73ed1fb...,6,config/ai2/ai2-games-all.json,2024-01-21 19:36:03.136035
213,Add-on fairy tale,A fairy tale that does not exist.,Gimmick,9e5ffe83e1eb131d9d2e3970e64a72d0736020e73ed1fb...,9,config/ai2-show/ai2-games-2024-05-18-z15.json,2024-05-18 11:43:45.481013
62,Add-on fairy tale,None,Continuation,9e5ffe83e1eb131d9d2e3970e64a72d0736020e73ed1fb...,3,config/ai2/ai2-games-all-2024-09.json,2024-09-09 19:24:04.543999
109,Add-on fairy tale,None,Continuation,9e5ffe83e1eb131d9d2e3970e64a72d0736020e73ed1fb...,5,config/ai2/ai2-games-all-2025-03-01.json,2025-03-01 21:21:30.830773


,name,prompt,category,uid,json_file,json_fname,created
214,Chain Death,"Occupation, location, ordinary object.",Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...,9,config/ai2-show/ai2-games-2024-05-18-z15.json,2024-05-18 11:43:45.481013
17,Chain Death,"Occupation, location, ordinary object.",Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...,1,config/ai2/ai2-games-all-2024-07.json,2024-07-08 12:15:15.936213
41,Chain Death,"Occupation, location, ordinary object\nAll wit...",Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...,2,config/ai2/ai2-games-all-2024-08.json,2024-08-05 19:34:47.276161
234,Chain Death,"Occupation, location, ordinary object\nAll wit...",Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...,10,config/ai2-show/ai2-games-2024-08-31.json,2024-08-27 18:12:43.116610
63,Chain Death,"Occupation, location, ordinary object\nAll wit...",Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...,3,config/ai2/ai2-games-all-2024-09.json,2024-09-09 19:24:04.543999
275,Chain Death,"Occupation, location, ordinary object\n\nAll w...",Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...,12,config/jam/open-cast-2024-11-games.json,2024-11-02 13:21:44.312375
88,Chain Death,"Occupation, location, ordinary object\n\nAll w...",Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...,4,config/ai2/ai2-games-all-2024-12.json,2024-11-25 21:49:53.005439
110,Chain Death,"Occupation, location, ordinary object\n\nAll w...",Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...,5,config/ai2/ai2-games-all-2025-03-01.json,2025-03-01 21:21:30.830773


,name,prompt,category,uid,json_file,json_fname,created
64,Da Doo Rap Rap,A one-syllable word,Musical,292d74803ca1045f053b84666dfd6a5a973fd31c0fae3b...,3,config/ai2/ai2-games-all-2024-09.json,2024-09-09 19:24:04.543999
111,Da Doo Rap Rap,A one-syllable word,Musical,292d74803ca1045f053b84666dfd6a5a973fd31c0fae3b...,5,config/ai2/ai2-games-all-2025-03-01.json,2025-03-01 21:21:30.830773


,name,prompt,category,uid,json_file,json_fname,created
148,Da Doo Ron Ron,A one-syllable word,,eae582cc8a879afbd22dcde81c49b1814aa7b632280ade...,6,config/ai2/ai2-games-all.json,2024-01-21 19:36:03.136035
18,Da Doo Ron Ron,A one-syllable word,Song,eae582cc8a879afbd22dcde81c49b1814aa7b632280ade...,1,config/ai2/ai2-games-all-2024-07.json,2024-07-08 12:15:15.936213
112,Da Doo Ron Ron,A one-syllable word,Song,eae582cc8a879afbd22dcde81c49b1814aa7b632280ade...,5,config/ai2/ai2-games-all-2025-03-01.json,2025-03-01 21:21:30.830773


...

,name,prompt,category,uid,json_file,json_fname,created
14,TV Box,What are four of your favorite TV channels?,,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,0,config/config/games.json,2023-11-13 22:34:52.292274
192,TV Box,What are four of your favorite TV channels?,,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,7,config/ai2/ai2-games.json,2023-11-13 22:39:24.773663
272,TV Box,What are four of your favorite TV channels?,,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,11,config/jam/jam-games.json,2023-11-13 22:39:44.303316
313,TV Box,What are four of your favorite TV channels?,,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,13,config/zmack/games.json,2023-11-15 16:13:59.377662
210,TV Box,What are four of your favorite TV channels?,NaN,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,8,config/ai2-show/ai2-games-2024-01-20.json,2024-01-21 18:48:13.619034
168,TV Box,What are four of your favorite TV channels?,,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,6,config/ai2/ai2-games-all.json,2024-01-21 19:36:03.136035
230,TV Box,What are 4 of your favorite \nTV channels?,???,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,9,config/ai2-show/ai2-games-2024-05-18-z15.json,2024-05-18 11:43:45.481013
37,TV Box,What are 4 of your favorite \nTV channels?,???,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,1,config/ai2/ai2-games-all-2024-07.json,2024-07-08 12:15:15.936213
58,TV Box,What are 4 of your favorite \nTV channels?,???,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,2,config/ai2/ai2-games-all-2024-08.json,2024-08-05 19:34:47.276161
251,TV Box,What are 4 of your favorite \nTV channels?,???,04643b71391301003f27d601c2b5115b9de9b0b68c2b12...,10,config/ai2-show/ai2-games-2024-08-31.json,2024-08-27 18:12:43.116610


,name,prompt,category,uid,json_file,json_fname,created
142,That Game,????,All Play,f95865da8d6a0adb483597832d6ed17afe2ad1beced0c2...,5,config/ai2/ai2-games-all-2025-03-01.json,2025-03-01 21:21:30.830773


,name,prompt,category,uid,json_file,json_fname,created
143,Three Monkeys,????,All Play,b0926c6519ac8d702fa87efef9320688258b945ab3273b...,5,config/ai2/ai2-games-all-2025-03-01.json,2025-03-01 21:21:30.830773


,name,prompt,category,uid,json_file,json_fname,created
169,Two lines,"Two lines,\nA location",,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,6,config/ai2/ai2-games-all.json,2024-01-21 19:36:03.136035
231,Two lines,"Two lines (a question, a statement).\nA location.",Gimmick,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,9,config/ai2-show/ai2-games-2024-05-18-z15.json,2024-05-18 11:43:45.481013
38,Two lines,"Two lines (a question, a statement).\nA location.",Gimmick,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,1,config/ai2/ai2-games-all-2024-07.json,2024-07-08 12:15:15.936213
59,Two lines,"Two lines (a question, a statement).\nA location.",Gimmick,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,2,config/ai2/ai2-games-all-2024-08.json,2024-08-05 19:34:47.276161
252,Two lines,"Two lines (a question, a statement).\nA location.",Gimmick,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,10,config/ai2-show/ai2-games-2024-08-31.json,2024-08-27 18:12:43.116610
85,Two lines,"Two lines (a question, a statement).\n+ A loca...",Limitation,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,3,config/ai2/ai2-games-all-2024-09.json,2024-09-09 19:24:04.543999
295,Two lines,"One question, one statement each.\n\n+ A locat...",Limitation,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,12,config/jam/open-cast-2024-11-games.json,2024-11-02 13:21:44.312375
106,Two lines,"One question, one statement each.\n\n+ A locat...",Limitation,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,4,config/ai2/ai2-games-all-2024-12.json,2024-11-25 21:49:53.005439
144,Two lines,"One question, one statement each.\n\n+ A locat...",Limitation,efcf3eaf27cc0a15ccb2b6055f459f390519c955528dcc...,5,config/ai2/ai2-games-all-2025-03-01.json,2025-03-01 21:21:30.830773


,name,prompt,category,uid,json_file,json_fname,created
15,World's Worst,"An occupation, animal, or celebrity",,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,0,config/config/games.json,2023-11-13 22:34:52.292274
193,World's Worst,"An occupation, animal, or celebrity",,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,7,config/ai2/ai2-games.json,2023-11-13 22:39:24.773663
273,World's Worst,"An occupation, animal, or celebrity",,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,11,config/jam/jam-games.json,2023-11-13 22:39:44.303316
314,World's Worst,"An occupation, animal, or celebrity",,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,13,config/zmack/games.json,2023-11-15 16:13:59.377662
211,World's Worst,"An occupation, animal, or celebrity",NaN,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,8,config/ai2-show/ai2-games-2024-01-20.json,2024-01-21 18:48:13.619034
170,World's Worst,"An occupation, animal, or celebrity",,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,6,config/ai2/ai2-games-all.json,2024-01-21 19:36:03.136035
232,World's Worst,"An occupation, animal, or celebrity.",All Play,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,9,config/ai2-show/ai2-games-2024-05-18-z15.json,2024-05-18 11:43:45.481013
39,World's Worst,"An occupation, animal, or celebrity.",All Play,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,1,config/ai2/ai2-games-all-2024-07.json,2024-07-08 12:15:15.936213
60,World's Worst,"An occupation, animal, or celebrity.",All Play,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,2,config/ai2/ai2-games-all-2024-08.json,2024-08-05 19:34:47.276161
253,World's Worst,"An occupation, animal, or celebrity.",All Play,a393da37e10dc0e60f8e91237ee4bac629b1129ec8c734...,10,config/ai2-show/ai2-games-2024-08-31.json,2024-08-27 18:12:43.116610


In [36]:
df_last_created = df.sort_values(by='created', ascending=False).drop_duplicates(subset='uid').sort_values('name')
df_last_created = df_last_created.set_index(keys='uid', drop=False)
df_last_created['exclude'] = [[] for i in range(df_last_created.shape[0])]
df_last_created['host_exclude'] = [[] for i in range(df_last_created.shape[0])]
df_last_created['host_include'] = [[] for i in range(df_last_created.shape[0])]
df_last_created.head(3)

,json_file,json_fname,game_key,created,name,nbr_players,nbr_audience,prompt,exclude,host_include,host_exclude,description,tips,category,uid
uid,,,,,,,,,,,,,,,
02ca6a55895dc6b0349024c92cbcecce71137e425d494b34b4447ff6cb618f55,5,config/ai2/ai2-games-all-2025-03-01.json,1,2025-03-01 21:21:30.830773,A Date with Me,8,0,"A date with me is like ....\n\nobject, animal,...",[],[],[],None,None,All Play,02ca6a55895dc6b0349024c92cbcecce71137e425d494b...
9e5ffe83e1eb131d9d2e3970e64a72d0736020e73ed1fb80b781d824e2f932d9,5,config/ai2/ai2-games-all-2025-03-01.json,2,2025-03-01 21:21:30.830773,Add-on fairy tale,4,0,None,[],[],[],None,None,Continuation,9e5ffe83e1eb131d9d2e3970e64a72d0736020e73ed1fb...
99307e9c5a831238501c742b7b069762078eb96b6648ef28f99cbcbe0f9115c5,5,config/ai2/ai2-games-all-2025-03-01.json,3,2025-03-01 21:21:30.830773,Chain Death,4,0,"Occupation, location, ordinary object\n\nAll w...",[],[],[],None,None,Guessing,99307e9c5a831238501c742b7b069762078eb96b6648ef...


In [52]:
p2json = root / 'config/ai2/ai2-games-all-ever.json'
coi = [
    'name', 'nbr_players','nbr_audience', 
    'prompt', 
    'exclude', 'host_include', 'host_exclude',
    'description', 'tips', 'category', 'uid'
    ]

dict_for_json = df_last_created.loc[:,coi].T.to_dict()

In [58]:
with open(p2json, 'w') as f:
    json.dump(dict_for_json, f, indent=4)

print(f"{p2json.name} saved")

ai2-games-all-ever.json saved
